In [1]:
# Imports
import pandas as pd
import numpy as np
import time
import psycopg2
from datetime import datetime
import statsmodels.api as sm
import pymssql

In [3]:
# Helper Functions

# Convert to integer
def int_convert(value):
    if pd.notna(value):
        try:
            return int(value)
        except (ValueError, TypeError):
            return None
    return None

# Handle Null Values
def handle_na(value):
    if pd.notna(value):
        return value
    else:
        return None
    
# Determine winner_ou
def get_winner_ou(total_score, over_under_line):
    if total_score > over_under_line:
        return 'over'
    elif total_score < over_under_line:
        return 'under'
    else:
        return 'push'

# Determine winner_line
def get_winner_line(score_home, score_away, spread_favorite, team_favorite_id, team_home):
    score_diff = score_home - score_away
    
    if team_favorite_id == team_home: # Home team is favorite (includes PICK games)
        if score_diff > -spread_favorite:
            return 'home'
        elif score_diff < -spread_favorite:
            return 'away'
        else:
            return 'push'
    else: # Away team is favorite
        if score_diff > spread_favorite:
            return 'home'
        elif score_diff < spread_favorite:
            return 'away'
        else:
            return 'push'

# Calculate commission based on bet amount
def calculate_commission(bet_amount):
    """
    - 10% on first $1,000
    - 8% on next $4,000
    - 6% on remaining amount
    """
    commission = 0
    
    # First $1,000
    if bet_amount <= 1000:
        commission = bet_amount * 0.10
    else:
        commission = 1000 * 0.10
        remaining = bet_amount - 1000
        
        # Next $4,000
        if remaining <= 4000:
            commission += remaining * 0.08
        else:
            commission += 4000 * 0.08
            remaining = remaining - 4000
            
            # Anything over $5,000
            commission += remaining * 0.06
            
    return round(commission, 2)

# Bet result
def determine_bet_result(bet_on, winner_line, winner_ou, team_home):    
    if bet_on == 'over':
        if winner_ou == 'over':
            return 'win'
        elif winner_ou == 'under':
            return 'loss'
        else:
            return 'push'
    elif bet_on == 'under':
        if winner_ou == 'under':
            return 'win'
        elif winner_ou == 'over':
            return 'loss'
        else:
            return 'push'
    elif bet_on == 'push':
        if winner_line == 'push':
            return 'win'
        else:
            return 'loss'
    else:  # Team bet
        cursor.execute("SELECT DISTINCT team_id FROM team_history WHERE name = %s;", (bet_on,))
        bet_team = cursor.fetchone()
        if bet_team:
            bet_team = bet_team[0]
            
            # Home or away team
            if bet_team == team_home:
                if winner_line == 'home':
                    return 'win'
                elif winner_line == 'away':
                    return 'loss'
                else:
                    return 'push'
            else:
                if winner_line == 'away':
                    return 'win'
                elif winner_line == 'home':
                    return 'loss'
                else:
                    return 'push'
        else:
            print(f"Error: Could not find team ID for bet on {bet_on}")
            return None

In [5]:
# Establish connections
conn = psycopg2.connect(
    database="iwdm", 
    user='dw_chancewiese',
    password='Spikeball2020',
    host='database-1.czsooswggscz.us-east-2.rds.amazonaws.com',
    port='5432'
)
cursor = conn.cursor()

pymssql_conn = pymssql.connect(
    server='stairwaytoheaven.usu.edu',
    user='5330user',
    password='pipelinesnow',
    database='ironwill'
)
pymssql_cursor = pymssql_conn.cursor()

In [4]:
# # For testing
# cursor.execute('''
# DROP TABLE IF EXISTS placed_bets;
# DROP TABLE IF EXISTS games;
# DROP TABLE IF EXISTS stadiums;
# DROP TABLE IF EXISTS weather_stations;
# DROP TABLE IF EXISTS teams;
# DROP TABLE IF EXISTS team_history;
# DROP TABLE IF EXISTS current_teams;
# DROP TABLE IF EXISTS customers;
# ''')
# conn.commit()

In [5]:
# Create Tables
# Customers
cursor.execute('''
CREATE TABLE IF NOT EXISTS customers (
    customer_id INTEGER PRIMARY KEY,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    age SMALLINT NOT NULL,
    customer_type VARCHAR(10) NOT NULL,
    customer_since SMALLINT NOT NULL,
    income INTEGER NOT NULL,
    household_size SMALLINT NOT NULL,
    mode_color VARCHAR(10) NOT NULL
    );''')
conn.commit()
print("Created customers table")

# Current Teams
cursor.execute('''
CREATE TABLE IF NOT EXISTS current_teams (
    team_id VARCHAR(5) PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    name_short VARCHAR(50) NOT NULL,
    team_id_pfr VARCHAR(5) NOT NULL,
    conference VARCHAR(5),
    division VARCHAR(20)
    );''')
conn.commit()
print("Created current teams table")

# Team History
cursor.execute('''
CREATE TABLE IF NOT EXISTS team_history (
    team_history_id SERIAL PRIMARY KEY,
    team_id VARCHAR(5) NOT NULL,
    name VARCHAR(100) NOT NULL,
    name_short VARCHAR(50) NOT NULL,
    team_id_pfr VARCHAR(5) NOT NULL,
    conference VARCHAR(5),
    division VARCHAR(20),
    FOREIGN KEY (team_id) REFERENCES current_teams (team_id)
    );''')
conn.commit()
print("Created team history table")

# Weather Stations
cursor.execute('''
CREATE TABLE IF NOT EXISTS weather_stations (
    weather_station_id CHAR(11) PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    state CHAR(2),
    country CHAR(2),
    latitude DECIMAL(8,5) NOT NULL,
    longitude DECIMAL(8,5) NOT NULL,
    elevation DECIMAL(5,1) NOT NULL
    );''')
conn.commit()
print("Created weather stations table")

# Stadiums
cursor.execute('''
CREATE TABLE IF NOT EXISTS stadiums (
    stadium_id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    stadium_open SMALLINT,
    stadium_close SMALLINT,
    stadium_type VARCHAR(20),
    street_address VARCHAR(100),
    city VARCHAR(50),
    state CHAR(2),
    postal_code VARCHAR(10),
    country CHAR(2),
    weather_station_id CHAR(11),
    stadium_weather_station_code INTEGER,
    stadium_weather_type VARCHAR(20),
    surface VARCHAR(50),
    FOREIGN KEY (weather_station_id) 
        REFERENCES weather_stations (weather_station_id)
    );''')
conn.commit()
print("Created stadiums table")

# Games
cursor.execute('''
CREATE TABLE IF NOT EXISTS games (
    game_id VARCHAR(50) PRIMARY KEY,
    schedule_date DATE NOT NULL,
    schedule_season SMALLINT NOT NULL,
    schedule_week SMALLINT NOT NULL,
    schedule_playoff BOOLEAN NOT NULL,
    playoff_type VARCHAR(20),
    team_id_home VARCHAR(5) NOT NULL,
    team_id_away VARCHAR(5) NOT NULL,
    score_home SMALLINT,
    score_away SMALLINT,
    team_id_favorite VARCHAR(5),
    spread_favorite DECIMAL(4,1),
    over_under_line DECIMAL(4,1),
    stadium_id INTEGER,
    stadium_neutral BOOLEAN NOT NULL,
    weather_temperature SMALLINT,
    weather_wind_mph SMALLINT,
    weather_humidity SMALLINT,
    weather_detail VARCHAR(100),
    winner_ou VARCHAR(10),
    winner_line VARCHAR(10),
    FOREIGN KEY (team_id_home) 
        REFERENCES current_teams (team_id),
    FOREIGN KEY (team_id_away) 
        REFERENCES current_teams (team_id),
    FOREIGN KEY (team_id_favorite) 
        REFERENCES current_teams (team_id),
    FOREIGN KEY (stadium_id) 
        REFERENCES stadiums (stadium_id)
    );''')
conn.commit()
print("Created games table")

# Placed Bets
cursor.execute('''
CREATE TABLE IF NOT EXISTS placed_bets (
    bet_id INTEGER PRIMARY KEY,
    customer_id INTEGER NOT NULL,
    game_id VARCHAR(50) NOT NULL,
    bet_amount INTEGER NOT NULL,
    bet_on VARCHAR(50) NOT NULL,
    bet_on_team_id VARCHAR(5),
    result VARCHAR(10),
    commission_amount DECIMAL(10,2),
    FOREIGN KEY (customer_id) 
        REFERENCES customers (customer_id),
    FOREIGN KEY (game_id) 
        REFERENCES games (game_id),
    FOREIGN KEY (bet_on_team_id) 
        REFERENCES current_teams (team_id)
    );''')
conn.commit()
print("Created placed_bets table")

Created customers table
Created current teams table
Created team history table
Created weather stations table
Created stadiums table
Created games table
Created placed_bets table


In [6]:
# Ingest Weather Stations

stadiums_df = pd.read_csv('nfl_stadiums.csv')
weather_stations_df = stadiums_df[['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION']]
df = weather_stations_df.dropna()

# Get current stations and convert to list
current_weather_stations = pd.read_sql("SELECT weather_station_id FROM weather_stations;", conn)
cws = current_weather_stations['weather_station_id'].tolist()

for x in df.index:
    weather_station_id = df['STATION'].loc[x]
    
    # Only proceed if station is new
    if weather_station_id not in cws:
        cws.append(weather_station_id) # Make sure a station isn't inserted twice
        name = df['NAME'].loc[x]
        latitude = df['LATITUDE'].loc[x]
        longitude = df['LONGITUDE'].loc[x]
        elevation = df['ELEVATION'].loc[x]

        # Get state and country codes
        name, state_country = name.split(', ')
        state, country = state_country.split(' ')

        cursor.execute('''
            INSERT INTO weather_stations (weather_station_id, name, state, country, latitude, longitude, elevation)
            VALUES (%s, %s, %s, %s, %s, %s, %s);
        ''', (weather_station_id, name, state, country, latitude, longitude, elevation))
        print(f"Inserted station {weather_station_id}: {name}")
        
conn.commit()

# Verify data
df = pd.read_sql("SELECT * FROM weather_stations", conn, index_col='weather_station_id')
print(f"Number of records: {len(cws)}\n",df)

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_7036/2151444998.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  current_weather_stations = pd.read_sql("SELECT weather_station_id FROM weather_stations;", conn)


Inserted station USW00094823: PITTSBURGH ASOS
Inserted station US1MOJC0028: KANSAS CITY 5.1 SE
Inserted station USC00410337: ARLINGTON SIX FLAGS
Inserted station USW00013881: CHARLOTTE DOUGLAS AIRPORT
Inserted station US1NYER0093: BUFFALO 1.5 W
Inserted station USC00238791: WEBSTER GROVES
Inserted station USW00023234: SAN FRANCISCO INTERNATIONAL AIRPORT
Inserted station US1WAKG0038: SEATTLE 3.5 NW
Inserted station USW00003871: CINCINNATI WEATHER SERVICE OFFICE CITY
Inserted station USW00014820: CLEVELAND HOPKINS INTERNATIONAL AIRPORT
Inserted station USW00023062: DENVER STAPLETON
Inserted station USW00093837: JACKSONVILLE NAS
Inserted station USC00186350: NATIONAL ARBORETUM DC
Inserted station USC00190860: BROCKTON
Inserted station USW00014734: NEWARK LIBERTY INTERNATIONAL AIRPORT
Inserted station USW00012839: MIAMI INTERNATIONAL AIRPORT
Inserted station USW00012842: TAMPA INTERNATIONAL AIRPORT
Inserted station USW00014898: GREEN BAY AUSTIN STRAUBEL INTERNATIONAL AIRPORT
Inserted stati

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_7036/2151444998.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM weather_stations", conn, index_col='weather_station_id')


In [7]:
# Ingest Stadiums

df = pd.read_csv('nfl_stadiums.csv')

# Get current stadiums and convert to list
current_stadiums = pd.read_sql("SELECT name FROM stadiums;", conn)
cs = current_stadiums['name'].tolist()

for x in df.index:
    name = df['stadium_name'].loc[x]
    
    if name not in cs:
        cs.append(name)  # Make sure a stadium isn't inserted twice
        location = df['stadium_location'].loc[x]
        stadium_open = df['stadium_open'].loc[x]
        stadium_close = df['stadium_close'].loc[x]
        stadium_type = df['stadium_type'].loc[x]
        address = df['stadium_address'].loc[x]
        stadium_weather_station_code = df['stadium_weather_station_code'].loc[x]
        weather_station_id = df['STATION'].loc[x]
        stadium_weather_type = df['stadium_weather_type'].loc[x]
        surface = df['stadium_surface'].loc[x]
        street_address = city = state = postal_code = country = None

        ### addresscleaning
        if pd.notna(location):
            if 'UK' in location:
                city, country = location.split(', ')
            elif 'MX' in str(stadium_weather_station_code):
                city = location.split(', ')[0]
                country = 'MX'
            elif 'Canada' in location:
                city = location.split(', ')[0]
                country = 'CA'
            elif 'Germany' in location:
                city = location.split(', ')[0]
                country = 'DE'
            elif ',' in location:  # US locations
                city, state = location.split(', ')
                country = 'US'
        # Parse address if it exists
        if pd.notna(address):
            if country == 'UK':
                parts = address.split(', ')
                street_address = parts[0]
                postal_code = parts[1].replace('London', '').strip()
            elif country == 'MX': # no clue how to handle this one
                parts = address.split(', ')
                street_address = parts[0]
            elif country == 'CA':
                parts = address.split(', ')
                street_address = parts[0]
                state = parts[2].split(' ')[0]
                postal_code = parts[2].split(' ')[1] + ' ' + parts[2].split(' ')[2]
            elif country == 'US':
                parts = address.split(', ')
                street_address = parts[0]
                postal_code = parts[-1].split(' ')[-1]
        
        ## Integer converting and null handling for insertion
        stadium_open = int_convert(stadium_open)
        stadium_close = int_convert(stadium_close)
        stadium_weather_station_code = int_convert(stadium_weather_station_code)
        weather_station_id = handle_na(weather_station_id)
        stadium_type = handle_na(stadium_type)
        stadium_weather_type = handle_na(stadium_weather_type)
                
                
        ## surface
        if pd.notna(surface):  # Check if surface exists before lowercase
            if 'grass' in surface.lower():
                surface = 'Grass'
            elif 'turf' in surface.lower():
                surface = 'Turf'
        else:
            surface = None
            
        ### Ingest
        cursor.execute('''
            INSERT INTO stadiums (name, stadium_open, stadium_close, stadium_type, street_address, city, state, postal_code, country, weather_station_id, stadium_weather_station_code, stadium_weather_type, surface)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
        ''', (name, stadium_open, stadium_close, stadium_type, street_address, city, state, postal_code, country, weather_station_id, stadium_weather_station_code, stadium_weather_type, surface))
        print(f"Inserted stadium: {name}")

conn.commit()

# Verify data
df = pd.read_sql("SELECT * FROM stadiums", conn, index_col='stadium_id')
print(f"Number of records: {len(cs)}\n", df)

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_7036/216549753.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  current_stadiums = pd.read_sql("SELECT name FROM stadiums;", conn)


Inserted stadium: Acrisure Stadium
Inserted stadium: Alamo Dome
Inserted stadium: Allegiant Stadium
Inserted stadium: Allianz Arena
Inserted stadium: Alltel Stadium
Inserted stadium: Alumni Stadium
Inserted stadium: Anaheim Stadium
Inserted stadium: Arrowhead Stadium
Inserted stadium: AT&T Stadium
Inserted stadium: Atlanta-Fulton County Stadium
Inserted stadium: Balboa Stadium
Inserted stadium: Bank of America Stadium
Inserted stadium: Bills Stadium
Inserted stadium: Busch Memorial Stadium
Inserted stadium: Caesars Superdome
Inserted stadium: Candlestick Park
Inserted stadium: CenturyLink Field
Inserted stadium: Cinergy Field
Inserted stadium: Cleveland Municipal Stadium
Inserted stadium: Cotton Bowl
Inserted stadium: Cowboys Stadium
Inserted stadium: Dolphin Stadium
Inserted stadium: Edward Jones Dome
Inserted stadium: Empower Field at Mile High
Inserted stadium: Estadio Azteca
Inserted stadium: EverBank Field
Inserted stadium: FedEx Field
Inserted stadium: Fenway Park
Inserted stadiu

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_7036/216549753.py:88: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM stadiums", conn, index_col='stadium_id')


In [8]:
# Ingest Teams
df = pd.read_csv('nfl_teams.csv')

# Insert pre-2002 data first
for x in df.index:
    team_id = df['team_id'].loc[x]
    name = df['team_name'].loc[x]
    name_short = df['team_name_short'].loc[x]
    team_id_pfr = df['team_id_pfr'].loc[x]
    conference = df['team_conference_pre2002'].loc[x]
    division = df['team_division_pre2002'].loc[x]

    # Clean data
    team_id = handle_na(team_id)
    name = handle_na(name)
    name_short = handle_na(name_short)
    team_id_pfr = handle_na(team_id_pfr)
    conference = handle_na(conference)
    division = handle_na(division)
    
    # Check if team_id exists
    cursor.execute('SELECT team_id FROM current_teams WHERE team_id = %s', (team_id,))
    exists = cursor.fetchone()
    
    if not exists:
        # Insert into current_teams only if it doesn't exist
        cursor.execute('''
            INSERT INTO current_teams (team_id, name, name_short, team_id_pfr, conference, division)
            VALUES (%s, %s, %s, %s, %s, %s);
        ''', (team_id, name, name_short, team_id_pfr, conference, division))
        
        # Insert into team_history
        cursor.execute('''
            INSERT INTO team_history 
            (team_id, name, name_short, team_id_pfr, conference, division)
            VALUES (%s, %s, %s, %s, %s, %s);
        ''', (team_id, name, name_short, team_id_pfr, conference, division))
        
        print(f"Inserted pre-2002 record for team: {name}")
    
# Check for changes and update if needed
for x in df.index:
    team_id = df['team_id'].loc[x]
    name = df['team_name'].loc[x]
    name_short = df['team_name_short'].loc[x]
    team_id_pfr = df['team_id_pfr'].loc[x]
    conference = df['team_conference'].loc[x]
    division = df['team_division'].loc[x]

    # Clean data
    team_id = handle_na(team_id)
    name = handle_na(name)
    name_short = handle_na(name_short)
    team_id_pfr = handle_na(team_id_pfr)
    conference = handle_na(conference)
    division = handle_na(division)

    # Get current values
    cursor.execute("SELECT name, name_short, team_id_pfr, conference, division FROM current_teams WHERE team_id = %s;", (team_id,))
    current = cursor.fetchone()
    
    if current:
        current_values = list(current)
        new_values = [name, name_short, team_id_pfr, conference, division]
        
        # Check if any values have changed
        if current_values != new_values:            
            cursor.execute('''
                INSERT INTO team_history 
                (team_id, name, name_short, team_id_pfr, conference, division)
                VALUES (%s, %s, %s, %s, %s, %s);
            ''', (team_id, name, name_short, team_id_pfr, conference, division))
            
            # Update current_teams
            cursor.execute('''
                UPDATE current_teams 
                SET name = %s, name_short = %s, team_id_pfr = %s, conference = %s, division = %s
                WHERE team_id = %s;
            ''', (name, name_short, team_id_pfr, conference, division, team_id))
            
            print(f"Updated record for team: {name}")

conn.commit()

# Verify data
print("\nCurrent Teams table:")
teams_df = pd.read_sql("SELECT * FROM current_teams", conn)
print(f"Number of current teams: {len(teams_df)}\n", teams_df)

print("\nTeam History table:")
history_df = pd.read_sql("SELECT * FROM team_history", conn)
print(f"Number of history records: {len(history_df)}\n", history_df)

Inserted pre-2002 record for team: Arizona Cardinals
Inserted pre-2002 record for team: Atlanta Falcons
Inserted pre-2002 record for team: Baltimore Colts
Inserted pre-2002 record for team: Baltimore Ravens
Inserted pre-2002 record for team: Boston Patriots
Inserted pre-2002 record for team: Buffalo Bills
Inserted pre-2002 record for team: Carolina Panthers
Inserted pre-2002 record for team: Chicago Bears
Inserted pre-2002 record for team: Cincinnati Bengals
Inserted pre-2002 record for team: Cleveland Browns
Inserted pre-2002 record for team: Dallas Cowboys
Inserted pre-2002 record for team: Denver Broncos
Inserted pre-2002 record for team: Detroit Lions
Inserted pre-2002 record for team: Green Bay Packers
Inserted pre-2002 record for team: Houston Oilers
Inserted pre-2002 record for team: Houston Texans
Inserted pre-2002 record for team: Jacksonville Jaguars
Inserted pre-2002 record for team: Kansas City Chiefs
Inserted pre-2002 record for team: Las Vegas Raiders
Inserted pre-2002 re

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_7036/2418123640.py:87: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  teams_df = pd.read_sql("SELECT * FROM current_teams", conn)
/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_7036/2418123640.py:91: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  history_df = pd.read_sql("SELECT * FROM team_history", conn)


In [9]:
# Ingest Customers

df = pd.read_sql("SELECT * FROM customer_table;", pymssql_conn)
# df = pd.read_csv('customer_data.csv') # Testing because mac won't connect to pymssql

# Get current customers and convert to list
current_customers = pd.read_sql('SELECT customer_id FROM customers;', conn)
cc = current_customers['customer_id'].tolist()

# Initialize timer to see how long ingestion takes. (just out of curiosity)
start_time = time.time()
last_check = start_time

# Process each customer
counter = 0
for x in df.index:
    customer_id = int_convert(df['customer_id'].loc[x])
    
    # Only proceed if customer is new
    if customer_id not in cc:
        cc.append(customer_id)
        
        customer_name = df['customer_name'].loc[x]
        age = int_convert(df['customer_age'].loc[x])
        customer_type = handle_na(df['customer_type'].loc[x])
        customer_since = int_convert(df['customer_since'].loc[x])
        income = int_convert(df['customer_income'].loc[x])
        household_size = int_convert(df['household_size'].loc[x])
        mode_color = handle_na(df['mode_color'].loc[x])
        
        # Name
        name_parts = customer_name.split(' ')
        if len(name_parts) == 2:
            first_name = name_parts[0]
            last_name = name_parts[1]
        else:
            first_name = name_parts[0]
            last_name = ' '.join(name_parts[1:])
        
        # Insert into customers table
        cursor.execute('''
            INSERT INTO customers 
            (customer_id, first_name, last_name, age, customer_type, customer_since, income, household_size, mode_color)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
        ''', (customer_id, first_name, last_name, age, customer_type, customer_since, income, household_size, mode_color))

        counter += 1
        if counter % 100 == 0 or counter == 1:  # Print progress every 100 records
            current_time = time.time()
            elapsed_since_last = current_time - last_check
            total_elapsed = current_time - start_time
    
            last_check_min = int(elapsed_since_last // 60)
            last_check_sec = int(elapsed_since_last % 60)
            total_min = int(total_elapsed // 60)
            total_sec = int(total_elapsed % 60)
    
            percent_complete = (counter / len(df)) * 100

            readable_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(current_time))
            if counter == 1:
                print(f"Started at {readable_time}")
            else:
                print(f"Processed {counter} customers ({percent_complete:.1f}%) | Last 100 customers: {last_check_min}:{last_check_sec:02d} | Total time: {total_min}:{total_sec:02d} | Finished at: {readable_time}")
            
            last_check = current_time

conn.commit()

# Final timing
end_time = time.time()
total_time = end_time - start_time
total_min = int(total_time // 60)
total_sec = int(total_time % 60)
print(f"\nTotal processing time: {total_min}:{total_sec:02d}")

# Verify data
print("\nCustomers table:")
customers_df = pd.read_sql("SELECT * FROM customers", conn)
print(f"Number of customers: {len(cc)}")
print(customers_df)

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_7036/4107959282.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  current_customers = pd.read_sql('SELECT customer_id FROM customers;', conn)


Started at 2024-12-07 12:02:38
Processed 100 customers (5.0%) | Last 100 customers: 0:04 | Total time: 0:04 | Finished at: 2024-12-07 12:02:43
Processed 200 customers (10.0%) | Last 100 customers: 0:04 | Total time: 0:08 | Finished at: 2024-12-07 12:02:47
Processed 300 customers (15.0%) | Last 100 customers: 0:04 | Total time: 0:13 | Finished at: 2024-12-07 12:02:52
Processed 400 customers (20.0%) | Last 100 customers: 0:04 | Total time: 0:17 | Finished at: 2024-12-07 12:02:56
Processed 500 customers (25.0%) | Last 100 customers: 0:04 | Total time: 0:21 | Finished at: 2024-12-07 12:03:00
Processed 600 customers (30.0%) | Last 100 customers: 0:04 | Total time: 0:26 | Finished at: 2024-12-07 12:03:05
Processed 700 customers (35.0%) | Last 100 customers: 0:04 | Total time: 0:30 | Finished at: 2024-12-07 12:03:09
Processed 800 customers (40.0%) | Last 100 customers: 0:04 | Total time: 0:34 | Finished at: 2024-12-07 12:03:13
Processed 900 customers (45.0%) | Last 100 customers: 0:04 | Total

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_7036/4107959282.py:79: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  customers_df = pd.read_sql("SELECT * FROM customers", conn)


Number of customers: 2000
      customer_id first_name last_name  age customer_type  customer_since  \
0               1   Coraline    Flores   23        online            2022   
1               2    Presley     Ortiz   41        online            2021   
2               3       Reid    Knight   40        online            2023   
3               4   Clarissa  Chandler   43         local            2023   
4               5      Isaac    Vaughn   23         phone            2023   
...           ...        ...       ...  ...           ...             ...   
1995         1996     Maxine      Pope   36         local            2020   
1996         1997    Camilla     Wolfe   46         local            2020   
1997         1998     Karina    Knight   36         local            2020   
1998         1999  Sebastian   Winters   55         local            2020   
1999         2000   Hadassah      Wade   29         local            2021   

      income  household_size mode_color  
0     1

In [10]:
# Ingest Games

# Read games data
df = pd.read_csv('spread_scores-2.csv')

# Filter for games in/after 2015
df = df[df['schedule_season'] >= 2015]

# Get current games and convert to list
current_games = pd.read_sql("SELECT game_id FROM games;", conn)
cg = current_games['game_id'].tolist()

TEAM_ID_CORRECTIONS = {'LV': 'LVR'}

# Initialize timer
start_time = time.time()
last_check = start_time

# Process each game
counter = 0
for x in df.index:
    # Build game_id 
    season = str(df['schedule_season'].loc[x])
    weeknum = df['schedule_week'].loc[x]
    
    if weeknum == "Wildcard":
        playoff_type = "Wildcard"
        weeknum = "19"
    elif weeknum == "Division":
        playoff_type = "Division"
        weeknum = "20"
    elif weeknum == "Conference":
        playoff_type = "Conference"
        weeknum = "21"
    elif weeknum == "Superbowl":
        playoff_type = "Superbowl"
        weeknum = "22"
    else:
        playoff_type = None
        weeknum = str(weeknum)
        
    if len(weeknum) < 2: 
        weeknum = "0" + weeknum

    weekstring = weeknum
    weeknum = int_convert(weeknum)
    
    cursor.execute(f"SELECT DISTINCT team_id FROM team_history WHERE name = '{df['team_home'].loc[x]}';")
    home_id = cursor.fetchone()[0]
    home_id = TEAM_ID_CORRECTIONS.get(home_id, home_id)
    
    cursor.execute(f"SELECT DISTINCT team_id FROM team_history WHERE name = '{df['team_away'].loc[x]}';")
    away_id = cursor.fetchone()[0]
    away_id = TEAM_ID_CORRECTIONS.get(away_id, away_id)
    
    game_id = f"{season}{weekstring}-{home_id}-{away_id}"
    
    # Only proceed if game is new
    if game_id not in cg:
        cg.append(game_id)
        
        # Get data and clean
        schedule_date = datetime.strptime(df['schedule_date'].loc[x], '%m/%d/%Y')
        schedule_season = int_convert(df['schedule_season'].loc[x])
        schedule_week = weeknum
        schedule_playoff = bool(df['schedule_playoff'].loc[x])
        score_home = int_convert(df['score_home'].loc[x])
        score_away = int_convert(df['score_away'].loc[x])
        team_favorite_id = df['team_favorite_id'].loc[x]
        spread_favorite = df['spread_favorite'].loc[x]
        over_under_line = df['over_under_line'].loc[x]
        stadium_name = df['stadium'].loc[x]
        stadium_neutral = bool(df['stadium_neutral'].loc[x])
        weather_temperature = int_convert(df['weather_temperature'].loc[x])
        weather_wind_mph = int_convert(df['weather_wind_mph'].loc[x])
        weather_humidity = int_convert(df['weather_humidity'].loc[x])
        weather_detail = handle_na(df['weather_detail'].loc[x])        


        # Find stadium
        stadium_id = None
        cursor.execute("SELECT stadium_id FROM stadiums WHERE name = %s;", (stadium_name,))
        stadium_result = cursor.fetchone()

        if stadium_result:
            stadium_id = stadium_result[0]
        else:
            # Insert new stadium and get its ID
            cursor.execute('''INSERT INTO stadiums (name) VALUES (%s);''', (stadium_name,))
            conn.commit()
            cursor.execute("SELECT stadium_id FROM stadiums WHERE name = %s;", (stadium_name,))
            stadium_id = cursor.fetchone()[0]
            print(f"Added new stadium: {stadium_name} with ID: {stadium_id}")
        
        # Handle PICK
        if team_favorite_id == 'PICK':
            team_favorite_id = home_id  # Use home team because PICK messes with FK. Will always be a push anyways
        else:
            team_favorite_id = TEAM_ID_CORRECTIONS.get(team_favorite_id, team_favorite_id)
        
        # Calculate winner_ou and winner_line
        total_score = score_home + score_away
        winner_ou = get_winner_ou(total_score, over_under_line)
        winner_line = get_winner_line(score_home, score_away, spread_favorite, team_favorite_id, home_id)

        # Insert into games table
        cursor.execute('''
            INSERT INTO games (
                game_id, schedule_date, schedule_season, schedule_week, 
                schedule_playoff, playoff_type, team_id_home, team_id_away, 
                score_home, score_away, team_id_favorite, spread_favorite,
                over_under_line, stadium_id, stadium_neutral, 
                weather_temperature, weather_wind_mph, weather_humidity,
                weather_detail, winner_ou, winner_line)
            VALUES (
                %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s, %s, %s);
        ''', (game_id, schedule_date, schedule_season, schedule_week,
              schedule_playoff, playoff_type, home_id, away_id,
              score_home, score_away, team_favorite_id, spread_favorite,
              over_under_line, stadium_id, stadium_neutral,
              weather_temperature, weather_wind_mph, weather_humidity,
              weather_detail, winner_ou, winner_line))

        counter += 1
        if counter % 100 == 0 or counter == 1:  # Print progress every 100 records
            current_time = time.time()
            elapsed_since_last = current_time - last_check
            total_elapsed = current_time - start_time
    
            last_check_min = int(elapsed_since_last // 60)
            last_check_sec = int(elapsed_since_last % 60)
            total_min = int(total_elapsed // 60)
            total_sec = int(total_elapsed % 60)
    
            percent_complete = (counter / len(df)) * 100

            readable_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(current_time))
            if counter == 1:
                print(f"Started at {readable_time}")
            else:
                print(f"Processed {counter} games ({percent_complete:.1f}%) | Last 100 games: {last_check_min}:{last_check_sec:02d} | Total time: {total_min}:{total_sec:02d} | Finished at: {readable_time}")
            
            last_check = current_time

conn.commit()

# Final timing
end_time = time.time()
total_time = end_time - start_time
total_min = int(total_time // 60)
total_sec = int(total_time % 60)
print(f"\nTotal processing time: {total_min}:{total_sec:02d}")

# Verify data
print("\nGames table:")
games_df = pd.read_sql("SELECT * FROM games;", conn)
print(f"Number of games: {len(cg)}")
print(games_df)

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_7036/573614324.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  current_games = pd.read_sql("SELECT game_id FROM games;", conn)


Started at 2024-12-07 12:04:07
Processed 100 games (4.1%) | Last 100 games: 0:17 | Total time: 0:17 | Finished at: 2024-12-07 12:04:25
Processed 200 games (8.1%) | Last 100 games: 0:17 | Total time: 0:35 | Finished at: 2024-12-07 12:04:42
Processed 300 games (12.2%) | Last 100 games: 0:17 | Total time: 0:52 | Finished at: 2024-12-07 12:05:00
Processed 400 games (16.3%) | Last 100 games: 0:17 | Total time: 1:10 | Finished at: 2024-12-07 12:05:17
Processed 500 games (20.3%) | Last 100 games: 0:17 | Total time: 1:27 | Finished at: 2024-12-07 12:05:35
Processed 600 games (24.4%) | Last 100 games: 0:17 | Total time: 1:45 | Finished at: 2024-12-07 12:05:52
Processed 700 games (28.5%) | Last 100 games: 0:17 | Total time: 2:03 | Finished at: 2024-12-07 12:06:10
Processed 800 games (32.5%) | Last 100 games: 0:17 | Total time: 2:20 | Finished at: 2024-12-07 12:06:27
Processed 900 games (36.6%) | Last 100 games: 0:17 | Total time: 2:37 | Finished at: 2024-12-07 12:06:45
Processed 1000 games (40.7

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_7036/573614324.py:157: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  games_df = pd.read_sql("SELECT * FROM games;", conn)


Number of games: 2458
             game_id schedule_date  schedule_season  schedule_week  \
0      201501-NE-PIT    2015-09-10             2015              1   
1      201501-ARI-NO    2015-09-13             2015              1   
2     201501-BUF-IND    2015-09-13             2015              1   
3      201501-CHI-GB    2015-09-13             2015              1   
4     201501-DAL-NYG    2015-09-13             2015              1   
...              ...           ...              ...            ...   
2453   202320-BUF-KC    2024-01-21             2023             20   
2454   202320-DET-TB    2024-01-21             2023             20   
2455   202321-BAL-KC    2024-01-28             2023             21   
2456   202321-SF-DET    2024-01-28             2023             21   
2457    202322-KC-SF    2024-02-11             2023             22   

      schedule_playoff playoff_type team_id_home team_id_away  score_home  \
0                False         None           NE          PI

In [7]:
# Ingest Bets

# Read betting data
df = pd.read_sql("SELECT * FROM betlog;", pymssql_conn)
# df = pd.read_csv('betting_data.csv') # Testing because mac won't connect to pymssql

# Get current bets and convert to list
current_bets = pd.read_sql("SELECT bet_id FROM placed_bets;", conn)
cb = current_bets['bet_id'].tolist()
GAME_ID_CORRECTIONS = {'LV': 'LVR', 'JAC': 'JAX'}

# Initialize timer
start_time = time.time()
last_check = start_time

# Process each bet
counter = 0
for x in df.index:
    # if counter >= 3000: # Testing
    #     break
    # if counter < 79000:  # Skip records we already processed
    #     counter += 1
    #     continue
        
    bet_id = int_convert(df['bet_id'].loc[x])
    
    # Only proceed if bet is new
    if bet_id not in cb:
        cb.append(bet_id)
        
        customer_id = int_convert(df['customer_id'].loc[x])

        # Fix game ids
        game_id = df['game_id'].loc[x]
        game_id_parts = game_id.split('-')
        season_week, team1, team2 = game_id_parts
        team1 = GAME_ID_CORRECTIONS.get(team1, team1)
        team2 = GAME_ID_CORRECTIONS.get(team2, team2)
        game_id = f"{season_week}-{team1}-{team2}"

        bet_amount = int_convert(df['bet_amount'].loc[x])
        bet_on = df['bet_on'].loc[x]

        if bet_on not in ['over', 'under', 'push']:
            cursor.execute("SELECT DISTINCT team_id FROM team_history WHERE name = %s;", (bet_on,))
            bet_team = cursor.fetchone()
            bet_on_team_id = bet_team[0] if bet_team else None
        else:
            bet_on_team_id = None
        
        # Get game details for bet result
        cursor.execute("SELECT winner_line, winner_ou, team_id_home FROM games WHERE game_id = %s;", (game_id,))
        game_details = cursor.fetchone()
        
        if game_details:
            winner_line, winner_ou, team_id_home = game_details
            
            # Get result and commission
            result = determine_bet_result(bet_on, winner_line, winner_ou, team_id_home)                
            commission_amount = calculate_commission(bet_amount)
            
            # Insert into placed_bets table
            cursor.execute('''
                INSERT INTO placed_bets (bet_id, customer_id, game_id, bet_amount, bet_on, bet_on_team_id, result, commission_amount)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s);
            ''', (bet_id, customer_id, game_id, bet_amount, bet_on, bet_on_team_id, result, commission_amount))
        else:
            print(f"Error: Could not find game details for game_id {game_id}")
            
    counter += 1
    if counter % 1000 == 0 or counter == 1:  # Print progress every 1000 records
        conn.commit()
        current_time = time.time()
        elapsed_since_last = current_time - last_check
        total_elapsed = current_time - start_time

        last_check_min = int(elapsed_since_last // 60)
        last_check_sec = int(elapsed_since_last % 60)
        total_min = int(total_elapsed // 60)
        total_sec = int(total_elapsed % 60)

        percent_complete = (counter / len(df)) * 100

        readable_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(current_time))
        if counter == 1:
            print(f"Started at {readable_time}")
        else:
            print(f"Processed {counter} bets ({percent_complete:.1f}%) | Last 1000 bets: {last_check_min}:{last_check_sec:02d} | Total time: {total_min}:{total_sec:02d} | Finished at: {readable_time}")
        
        last_check = current_time

conn.commit()

# Final timing
end_time = time.time()
total_time = end_time - start_time
total_min = int(total_time // 60)
total_sec = int(total_time % 60)
print(f"\nTotal processing time: {total_min}:{total_sec:02d}")

# Verify data
print("\nBets table:")
bets_df = pd.read_sql("SELECT * FROM placed_bets;", conn)
print(f"Number of bets: {len(cb)}")
print(bets_df)

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_18178/200328037.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  current_bets = pd.read_sql("SELECT bet_id FROM placed_bets;", conn)


Processed 80000 bets (63.5%) | Last 1000 bets: 2:52 | Total time: 2:52 | Finished at: 2024-12-08 01:07:20
Processed 81000 bets (64.3%) | Last 1000 bets: 2:52 | Total time: 5:44 | Finished at: 2024-12-08 01:10:13
Processed 82000 bets (65.1%) | Last 1000 bets: 2:51 | Total time: 8:36 | Finished at: 2024-12-08 01:13:05
Processed 83000 bets (65.8%) | Last 1000 bets: 2:50 | Total time: 11:27 | Finished at: 2024-12-08 01:15:56
Processed 84000 bets (66.6%) | Last 1000 bets: 2:52 | Total time: 14:19 | Finished at: 2024-12-08 01:18:48
Processed 85000 bets (67.4%) | Last 1000 bets: 3:01 | Total time: 17:20 | Finished at: 2024-12-08 01:21:49
Processed 86000 bets (68.2%) | Last 1000 bets: 3:01 | Total time: 20:21 | Finished at: 2024-12-08 01:24:50
Processed 87000 bets (69.0%) | Last 1000 bets: 2:52 | Total time: 23:14 | Finished at: 2024-12-08 01:27:43
Processed 88000 bets (69.8%) | Last 1000 bets: 2:53 | Total time: 26:07 | Finished at: 2024-12-08 01:30:36
Processed 89000 bets (70.6%) | Last 1000

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_18178/200328037.py:103: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  bets_df = pd.read_sql("SELECT * FROM placed_bets;", conn)


Number of bets: 126047
        bet_id  customer_id        game_id  bet_amount               bet_on  \
0            1            1  202301-KC-DET        8500        Detroit Lions   
1            2            7  202301-KC-DET         350                under   
2            3            8  202301-KC-DET       10000   Kansas City Chiefs   
3            4           23  202301-KC-DET         200                under   
4            5           24  202301-KC-DET         400                 over   
...        ...          ...            ...         ...                  ...   
126042  126043         1988   202322-KC-SF        2500  San Francisco 49ers   
126043  126044         1989   202322-KC-SF         100  San Francisco 49ers   
126044  126045         1991   202322-KC-SF        1700   Kansas City Chiefs   
126045  126046         1995   202322-KC-SF         225                 push   
126046  126047         1999   202322-KC-SF        1800   Kansas City Chiefs   

       bet_on_team_id result

In [8]:
# Test Queries

# 1
tables = ['customers', 'placed_bets', 'games', 'team_history', 'current_teams', 'stadiums', 'weather_stations']
for table in tables:
    query = f"SELECT * FROM {table} LIMIT 5;"
    df = pd.read_sql(query, conn)
    print(f"\nFirst 5 rows from {table}:\n", df)


First 5 rows from customers:
    customer_id first_name last_name  age customer_type  customer_since  \
0            1   Coraline    Flores   23        online            2022   
1            2    Presley     Ortiz   41        online            2021   
2            3       Reid    Knight   40        online            2023   
3            4   Clarissa  Chandler   43         local            2023   
4            5      Isaac    Vaughn   23         phone            2023   

   income  household_size mode_color  
0  116000               1        red  
1  111000               1       blue  
2   55000               1      green  
3   32000               2     orange  
4   46000               1       blue  

First 5 rows from placed_bets:
    bet_id  customer_id        game_id  bet_amount              bet_on  \
0       1            1  202301-KC-DET        8500       Detroit Lions   
1       2            7  202301-KC-DET         350               under   
2       3            8  202301-KC-DET 

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_18178/226558871.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_18178/226558871.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_18178/226558871.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel


First 5 rows from team_history:
    team_history_id team_id               name name_short team_id_pfr  \
0                1     ARI  Arizona Cardinals  Cardinals         CRD   
1                2     ATL    Atlanta Falcons    Falcons         ATL   
2                3     IND    Baltimore Colts      Colts         CLT   
3                4     BAL   Baltimore Ravens     Ravens         RAV   
4                5      NE    Boston Patriots   Patriots         NWE   

  conference     division  
0        NFC     NFC West  
1        NFC     NFC West  
2        AFC     AFC East  
3        AFC  AFC Central  
4        AFC         None  

First 5 rows from current_teams:
   team_id                name name_short team_id_pfr conference  division
0     BUF       Buffalo Bills      Bills         BUF        AFC  AFC East
1     DAL      Dallas Cowboys    Cowboys         DAL        NFC  NFC East
2     DEN      Denver Broncos    Broncos         DEN        AFC  AFC West
3      KC  Kansas City Chiefs     

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_18178/226558871.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_18178/226558871.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_18178/226558871.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [9]:
# 2a
query = """
WITH commission_totals AS (
    SELECT c.customer_id, SUM(commission_amount) AS total_commission
    FROM customers c
    JOIN placed_bets pb ON c.customer_id = pb.customer_id
    GROUP BY c.customer_id
    HAVING SUM(commission_amount) > 20000
)
SELECT 
    COUNT(*) AS customers_over_20k,
    (SELECT COUNT(*) FROM customers) AS total_customers,
    CAST(COUNT(*) AS DECIMAL) / (SELECT COUNT(*) FROM customers) * 100 AS percentage
FROM commission_totals;
"""
df = pd.read_sql(query, conn)
print("Summary of high-commission customers:\n", df)

# 2b
query = """
SELECT c.first_name, c.last_name, SUM(commission_amount) AS total_commission
FROM customers c
JOIN placed_bets pb ON c.customer_id = pb.customer_id
GROUP BY c.customer_id, first_name, last_name
HAVING SUM(commission_amount) > 20000
ORDER BY total_commission DESC
LIMIT 20;
"""
df = pd.read_sql(query, conn)
print("\nTop 20 commission payers:\n", df)

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_18178/2919843573.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_18178/2919843573.py:29: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Summary of high-commission customers:
    customers_over_20k  total_customers  percentage
0                 236             2000        11.8

Top 20 commission payers:
    first_name last_name  total_commission
0        Gary   McClain          160114.5
1     Matthew     Booth          147968.5
2      Vaughn     Ortiz          126964.0
3      Bailey    Sparks          123294.0
4       Alexa   Mendoza          122644.0
5      Violet    Tanner          110888.0
6       Nadia     Frank          107458.0
7     Harmoni   Burnett           98722.5
8       Simon     Cohen           98533.5
9       Lucas    Wagner           96160.5
10      Clark    Jordan           93224.5
11      April    Martin           93212.5
12       Kate     Noble           89231.0
13       Lara   Hoffman           89225.0
14    Winston     Short           86980.0
15     Chanel      Pope           86525.5
16     Austin    Flores           86287.5
17     Warren   Barnett           85294.0
18    Adeline    Conner          

In [10]:
# 3
query = """
WITH bet_stats AS (
    SELECT 
        c.customer_id, first_name, last_name, COUNT(*) AS total_bets,
        COUNT(CASE WHEN pb.result = 'win' THEN 1 END) AS wins,
        SUM(CASE 
            WHEN pb.result = 'win' THEN pb.bet_amount
            WHEN pb.result = 'loss' THEN -pb.bet_amount
            WHEN pb.result = 'push' THEN 0 END) AS total_winnings
    FROM customers c
    JOIN placed_bets pb ON c.customer_id = pb.customer_id
    GROUP BY c.customer_id, c.first_name, c.last_name
    HAVING COUNT(*) >= 6
)
SELECT 
    first_name, last_name, total_bets, wins,
    ROUND((CAST(wins AS DECIMAL) / total_bets * 100), 2) AS win_percentage,
    total_winnings
FROM bet_stats
ORDER BY win_percentage DESC, total_winnings DESC
LIMIT 10;
"""
df = pd.read_sql(query, conn)
print("Top 10 luckiest bettors:\n", df)

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_18178/983587097.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Top 10 luckiest bettors:
   first_name last_name  total_bets  wins  win_percentage  total_winnings
0  Anastasia    Powell           6     6          100.00             825
1      Caleb  Reynolds         126   119           94.44           91475
2      Grace      Ford          11    10           90.91           13550
3      Henry  Anderson           9     8           88.89            3300
4    Valerie   Wheeler           8     7           87.50             850
5  Alexander   Sherman           7     6           85.71            4225
6       John       Lee          13    11           84.62           14900
7    Ezekiel  Martinez          13    11           84.62           11550
8    Sabrina    Wilson          12    10           83.33           41700
9    Houston    Foster           6     5           83.33           10300


In [11]:
# 4
query = """
SELECT 
    first_name, last_name, COUNT(*) AS total_bets,
    COUNT(CASE WHEN result = 'win' THEN 1 END) AS wins,
    SUM(CASE 
        WHEN result = 'win' THEN -(bet_amount * 2)
        WHEN result = 'loss' THEN bet_amount + commission_amount
        WHEN result = 'push' THEN commission_amount END) AS net_lost_for_company
FROM customers c
JOIN placed_bets pb ON c.customer_id = pb.customer_id
GROUP BY c.customer_id, first_name, last_name
ORDER BY net_lost_for_company
LIMIT 20
"""
df = pd.read_sql(query, conn)
print("20 costliest customers for the sports book:\n", df)

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_18178/2361486645.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


20 costliest customers for the sports book:
    first_name last_name  total_bets  wins  net_lost_for_company
0      Bailey    Sparks         180   100            -1206521.0
1        Kate     Noble         148    89            -1159675.0
2      Chanel      Pope         145    82             -829695.0
3     Adeline    Conner         194   112             -825578.0
4        Lara   Hoffman         119    65             -821873.0
5        Owen  Garrison          77    48             -810180.0
6     Matthew     Booth         250   117             -808651.5
7     Jackson   Fischer         137    79             -775182.0
8     Winston     Short         206   111             -766862.5
9        Ryan      Lane          81    49             -765602.0
10     Lennox      Bean         122    66             -756874.0
11     Vaughn     Ortiz         232   105             -724253.0
12      Clark    Jordan         163    85             -707735.0
13      Tiana  Lawrence         121    81             -6881

In [13]:
# 5 
query = """
WITH weekly_games AS (
    SELECT 
        schedule_week, COUNT(*) AS total_games,
        COUNT(CASE WHEN 
            (winner_line = 'home' AND spread_favorite > 0) OR 
            (winner_line = 'away' AND spread_favorite < 0) THEN 1 END) AS winners,
        COUNT(CASE WHEN 
            (winner_line = 'away' AND spread_favorite > 0) OR
            (winner_line = 'home' AND spread_favorite < 0) THEN 1 END) AS losers
    FROM games
    WHERE schedule_season = 2023 AND schedule_playoff = false
    GROUP BY schedule_week
)
SELECT 
    schedule_week, total_games, winners, losers,
    ROUND((CAST(winners AS DECIMAL) / total_games * 100), 2) AS winner_percentage,
    ROUND((CAST(losers AS DECIMAL) / total_games * 100), 2) AS loser_percentage
FROM weekly_games
ORDER BY schedule_week;
"""
df = pd.read_sql(query, conn)
print("Weekly house performance for 2023 season:\n", df)

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_18178/3296380341.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Weekly house performance for 2023 season:
     schedule_week  total_games  winners  losers  winner_percentage  \
0               1           16       12       4              75.00   
1               2           16        7       7              43.75   
2               3           16        7       8              43.75   
3               4           16        7       8              43.75   
4               5           14        8       6              57.14   
5               6           15        6       9              40.00   
6               7           13        3      10              23.08   
7               8           16        5       8              31.25   
8               9           14        6       8              42.86   
9              10           14        8       6              57.14   
10             11           14        8       5              57.14   
11             12           16        9       7              56.25   
12             13           13        9       4

In [14]:
# 6
query = """
WITH unique_bet_teams AS (
    SELECT DISTINCT bet_on AS team_name, bet_on_team_id AS team_id
    FROM placed_bets 
    WHERE bet_on_team_id IS NOT NULL
),
team_games AS (
    SELECT 
        ubt.team_name, g.game_id, score_home, score_away, winner_line, spread_favorite, team_id_favorite,
        CASE WHEN team_id_home = ubt.team_id THEN 'home'
             ELSE 'away' END AS home_or_away
    FROM unique_bet_teams ubt
    JOIN games g ON ubt.team_id = g.team_id_home OR ubt.team_id = g.team_id_away
    WHERE schedule_season = 2023
),
team_records AS (
    SELECT 
        team_name,
        COUNT(*) AS games_played,
        COUNT(CASE WHEN 
            (home_or_away = 'home' AND score_home > score_away) OR
            (home_or_away = 'away' AND score_away > score_home) 
            THEN 1 END) AS wins,
        COUNT(CASE WHEN 
            (home_or_away = 'home' AND score_home < score_away) OR
            (home_or_away = 'away' AND score_away < score_home) 
            THEN 1 END) AS losses,
        COUNT(CASE WHEN home_or_away = winner_line THEN 1 END) AS beat_spread
    FROM team_games
    GROUP BY team_name
),
betting_counts AS (
    SELECT 
        ubt.team_name,
        COUNT(CASE WHEN pb.bet_on = ubt.team_name THEN 1 END) AS bets_for,
        COUNT(CASE WHEN 
            pb.bet_on_team_id IS NOT NULL AND
            pb.bet_on != ubt.team_name
            THEN 1 END) AS bets_against
    FROM unique_bet_teams ubt
    LEFT JOIN games g ON ubt.team_id = g.team_id_home 
                     OR ubt.team_id = g.team_id_away
    LEFT JOIN placed_bets pb ON g.game_id = pb.game_id
    WHERE schedule_season = 2023
    GROUP BY ubt.team_id, ubt.team_name
)
SELECT 
    r.team_name, wins, losses, beat_spread,
    COALESCE(bets_for, 0) AS bets_for,
    COALESCE(bets_against, 0) AS bets_against
FROM team_records r
LEFT JOIN betting_counts b ON r.team_name = b.team_name
ORDER BY r.team_name;
"""
df = pd.read_sql(query, conn)
print("Per-team betting analysis for 2023 season:\n", df)

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_18178/2418816236.py:56: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Per-team betting analysis for 2023 season:
                 team_name  wins  losses  beat_spread  bets_for  bets_against
0       Arizona Cardinals     4      13            8      3058          3025
1         Atlanta Falcons     7      10            5      3020          2917
2        Baltimore Ravens    14       5           12      3465          3476
3           Buffalo Bills    12       7            8      3368          3341
4       Carolina Panthers     2      15            4      2956          2939
5           Chicago Bears     7      10            8      3003          2889
6      Cincinnati Bengals     9       8            8      2906          3057
7        Cleveland Browns    11       7            9      3223          3186
8          Dallas Cowboys    12       6           10      3195          3126
9          Denver Broncos     8       9            6      3001          2935
10          Detroit Lions    14       6           14      3533          3639
11      Green Bay Packers    10 

In [11]:
# Test Analysis

# Query to calculate customer value and create features
query = """
WITH bet_values AS (
    SELECT 
        customer_id,
        SUM(bet_amount) AS total_bet,
        SUM(commission_amount) as total_commission,
        COUNT(*) as num_bets,
        COUNT(CASE WHEN result = 'win' THEN 1 END) as num_wins,
        SUM(CASE 
            WHEN result = 'win' THEN -(bet_amount * 2) + commission_amount
            WHEN result = 'loss' THEN bet_amount + commission_amount
            WHEN result = 'push' THEN commission_amount
        END) AS bet_outcomes
    FROM placed_bets
    GROUP BY customer_id
)
SELECT 
    c.customer_id,
    age,
    income,
    household_size, 
    customer_type,
    customer_since,
    mode_color,
    first_name,
    last_name,
    num_bets,
    num_wins,
    CASE WHEN customer_type = 'local' THEN 1 ELSE 0 END as type_local,
    CASE WHEN customer_type = 'online' THEN 1 ELSE 0 END as type_online,
    CASE WHEN customer_type = 'phone' THEN 1 ELSE 0 END as type_phone,
    total_commission + bet_outcomes as customer_value
FROM customers c
LEFT JOIN bet_values bv ON c.customer_id = bv.customer_id;
"""
df = pd.read_sql(query, conn)

# Feature engineering
df['name_length'] = df['first_name'].str.len() + df['last_name'].str.len()
df['years_customer'] = 2024 - df['customer_since']

# A: Initial correlation matrix with all variables
initial_predictors = ['age', 'income', 'household_size', 'type_local', 'type_online', 'type_phone', 'years_customer', 'name_length']
corr_matrix = df[initial_predictors].corr()
print("\nInitial Correlation Matrix:\n", corr_matrix)
    
# B: First regression
first_predictors = ['age', 'income', 'household_size', 'type_local']
X = df[first_predictors]
y = df['customer_value']
X = sm.add_constant(X)
model1 = sm.OLS(y, X).fit()
print("\nFirst Model Results:\n", model1.summary())

# C: Second regression
second_predictors = ['income', 'years_customer', 'household_size', 'name_length', 'type_local']
X2 = df[second_predictors]
y = df['customer_value']
X2 = sm.add_constant(X2)
model2 = sm.OLS(y, X2).fit()
print("\nSecond Model Results:\n", model2.summary())

print("""
Regression Analysis Summary:
--------------------------
Correlation Analysis:
- Collinearity Issues:
   * High collinearity between customer type dummy variables
- Correlation Values:
   * type_online and type_local: -0.773
   * type_phone and type_local: -0.456
   * type_phone and type_online: -0.212
   
First Regression Model:
- Independent variables: age, income, household_size, type_local
- Adjusted R-squared: 0.050
- All variables except age are significant at p<0.05 (age: p=0.967)

Second Regression Model (Adjusted):
- Independent variables: income, years_customer, household_size, name_length, type_local
- Adjusted R-squared: 0.072
- All variables except name_length are significant at p<0.05 (name_length: p=0.191)
- Achieved higher adjusted R-squared by:
   * Adding engineered variables years_customer and name_length
   * Removing insignificant age variable
""")

/var/folders/96/kzlzzx2556jf68c57bc_gc180000gn/T/ipykernel_26287/3403901508.py:39: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)



Initial Correlation Matrix:
                      age    income  household_size  type_local  type_online  \
age             1.000000 -0.003367       -0.003167    0.003520     0.012831   
income         -0.003367  1.000000       -0.010054    0.020711    -0.011735   
household_size -0.003167 -0.010054        1.000000   -0.017376     0.012117   
type_local      0.003520  0.020711       -0.017376    1.000000    -0.772538   
type_online     0.012831 -0.011735        0.012117   -0.772538     1.000000   
type_phone     -0.023397 -0.015428        0.009761   -0.456359    -0.212437   
years_customer  0.020605  0.005408       -0.009284    0.056665    -0.027774   
name_length     0.006064 -0.045654       -0.047149    0.001623    -0.020042   

                type_phone  years_customer  name_length  
age              -0.023397        0.020605     0.006064  
income           -0.015428        0.005408    -0.045654  
household_size    0.009761       -0.009284    -0.047149  
type_local       -0.456359

In [16]:
# Close cursors and connections
cursor.close()
conn.close()

pymssql_cursor.close() 
pymssql_conn.close()